# Nested Sampling Algorithm

- Jeffrey's prior for $D_0$
- Uniform Priors for $\alpha$, $f$ and $\sigma^2_{mn}$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import scipy.stats as stats
import warnings

In [2]:
def jeff(u,magmin, magmax):
    return magmin*np.exp(u*np.log(magmax/magmin))
def uni(x, magmin, magmax):
    return 1/(magmax, magmin)


## likelihood calc function

In [3]:
@jit
def noisylikelihoodcalc(posn, DT, D, ALPHA, F =0 , varmn = 0):
    # time series
    n = len(posn)

    #init value
    logl = 0
    k_BT =1
    

    # Intial values of the variance mean both noisy and clean
    mob = D * (np.abs(posn[0]) ** ALPHA)
    std_dev_noise = 2 * mob * DT + 2 * varmn
    noisy_mean = posn[0] + (mob * F + k_BT * (ALPHA * D * ((np.abs(posn[0]) ** (ALPHA - 1)) * (np.sign(posn[0]))))) * DT
    
    # log likelihood at 1st step
    logl = logl - ((posn[1] - noisy_mean) ** 2) / (2 * std_dev_noise) - np.log(np.sqrt(2 * np.pi * std_dev_noise))
    
    for i in range(2,n):
        # Mobility at the step
        mob = D * np.abs(posn[i - 1]) ** ALPHA

        # clean variance at the jth step
        std_dev = 2 * mob * DT

        # clean mean position at the jth time step
        mean_dist = posn[i - 1] + (mob * F + ALPHA * D * (np.abs(posn[i - 1]) ** (ALPHA - 1)) * (np.sign(posn[i - 1]))) * DT

        # noisy mean position at the jth step
        noisy_mean = mean_dist - varmn/(std_dev_noise) * (posn[i-1] - noisy_mean)

        # noisy variance at the jth posn
        std_dev_noise = std_dev + varmn * (2 - varmn / std_dev_noise)

        # calculation of log likelihood
        logl = logl - ((posn[i] - noisy_mean) ** 2) / (2 * std_dev_noise) - 0.5 * np.log(2 * np.pi * std_dev_noise)
    
    return logl

## Model for calculating Evidence and with consider Model parameters

Instead of defining different functions for different models we could use the theta_max and theta_min vectors to define the parameters properly.

In [4]:
@jit
def Evidence(data, theta, npoints, tol):
    logZ = 0
    walker = walkergen(npoints, theta_max, theta_min)
    for i in range(0,npoints):
        ll[i] = noisylikelihoodcalc(p_data, 1, walker[i][0], walker[i][1], walker[i][2], walker[i][3])
    
    

In [5]:
@jit
def walkergen(n, tmax, tmin):
    x = np.zeros((n,4))
    for i in range(0,n):
        for j in range(0,4):
            x[i][j] = np.random.uniform(tmin[j],tmax[j])
    return x

In [6]:
# @jit 
def logsum(a, b):
    return max(a,b) + np.log(1+ np.exp(-np.abs(a-b)))

## $\theta_{max}$ and $\theta_{min}$ vectors

we use this to define the priors

In [7]:
D_min = 10**(-4)
D_max = 10**2
# D_min = 0.
# D_max = 1.


f_min = -1.
f_max = 1.

alpha_min = -2.
alpha_max = 2.

varmn_min = 1.
varmn_max = 100.

theta_max_pc = np.array([D_max, alpha_max, f_max, 0])
theta_min_pc = np.array([D_min, alpha_min, f_min, 0])

theta_max_fc = np.array([D_max, alpha_max, 0, 0])
theta_min_fc = np.array([D_min, alpha_min, 0, 0])

theta_max_pmn = np.array([D_max, alpha_max, f_max, varmn_max])
theta_min_pmn = np.array([D_min, alpha_min, f_min, varmn_min])

theta_max_fmn = np.array([D_max, alpha_max, 0, varmn_max])
theta_min_fmn = np.array([D_min, alpha_min, 0, varmn_min])


In [8]:
p_data = np.array([50,46.743,46.751,40.915,45.066,50.56,44.448,40.518,40.145,39.551,42.887,40.218,44.399,38.416,36.626,35.867,30.252,28.843,26.412,27.815,28.166,32.899,34.738,33.237,24.448,22.771,19.285,24.966,23.912,23.839,23.83,22.253,15.661,20.156,20.166,16.9,15.495,18.734,18.234,17.823,16.052,11.424,9.4223,7.3746,7.2549,7.2847,6.7366,7.7324,6.8572,7.034,5.3564,3.7969,3.4073,4.3653,4.56,4.8884,3.8021,2.9259,3.5701,4.0879,2.3471,0.95298,1.7231,1.7968,1.753,2.1651,2.1354,2.404,2.9188,1.1974,1.3358,1.428,1.8476,2.1991,1.1013,1.3227,0.71634,0.68376,0.89795,1.6167,2.0934,2.4119,1.0693,1.2777,2.2396,0.30283,0.64551,-0.44319,-0.79814,-0.82102,-2.0478,-1.6834,-1.8176,-1.3092,-1.2355,-1.8231,-1.8543,-2.1829,-4.2785,-4.2926])

# MCMC walker

Tried my hand at coding a MCMC walker using the Metropolis Hastings algo. The proposal function is a standard gaussian. To make it compatible for different models use can use different step sizes and then set the step sizes for the non fixed parameters to be the same and step size for the fixed parameter to be 0. So the walker is also constrained to walk over a surface that has fixed parameter of the model to be fixed. Setting the proposal function like this means we can define an easier code snippet without having to define multiple methods for different models and then it would be easier to define.

We could also go an easier way and then generate the walkers only in the constrained surface

Have to try both, but have a feeling that the secind method will be more efficient.

In [46]:
# @jit
def MCMCwalker(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in range(0,4):
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar :
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [45]:
# @jit
def MCMCwalkerfmn(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in [0,1,3]:
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [47]:
# @jit
def MCMCwalkerpc(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in range(0,3):
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar :
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [48]:
# @jit
def MCMCwalkerfc(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in [0,1]:
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar :
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

# Nested Sampling Code starts here

In [57]:
# steps for the random walkers in the MCMC algo
Nsteps = 15000

# no of walkers considered
npoints = 100

# init likelihood of pull, clean and free, clean models
llpc = np.zeros(npoints)
llfc = np.zeros(npoints)
llpmn = np.zeros(npoints)
llfmn = np.zeros(npoints)

# walkers pull clean and walker free clean
walkers_pc = np.array(walkergen(npoints, theta_max_pc,theta_min_pc))
walkers_fc = np.array(walkergen(npoints, theta_max_fc,theta_min_fc))
walkers_pmn = np.array(walkergen(npoints, theta_max_pmn,theta_min_pmn))
walkers_fmn = np.array(walkergen(npoints, theta_max_fmn,theta_min_fmn))

# walkers_pc = walkers_pmn.copy()
# # walkers_pmn = walkers_pmn
# walkers_fc = walkers_pmn.copy()
# walkers_fmn = walkers_pmn.copy()
# for i in range(0, npoints):
#     walkers_pc[i][3] = 0
#     walkers_fc[i][3] = 0
#     walkers_fc[i][2] = 0
#     walkers_fmn[i][2] = 0

walkers_pmn, walkers_fc, walkers_fmn, walkers_pc;

In [58]:
# likelihood  of Mpc and Mfc
for i in range(0,npoints):
    llpc[i] = noisylikelihoodcalc(p_data, 1, *walkers_pc[i])
    llfc[i] = noisylikelihoodcalc(p_data, 1, *walkers_fc[i])
    llpmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_pmn[i])
    llfmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_fmn[i])


# wt = 1/(npoints + 1)
logwt = -np.log(npoints + 1)

#Zpc is Z pull clean
#Zfc is Z free clean

# these are the remainders in the jth iteration
logZpcrem = np.log(0)
logZfcrem = np.log(0)
logZpmnrem = np.log(0)
logZfmnrem = np.log(0)

# these are the Log Evidences for the respective models
logZpc = np.log(0)
logZfc = np.log(0)
logZpmn = np.log(0)
logZfmn = np.log(0)


logp_pc = min(llpc) + logwt
logp_fc = min(llfc) + logwt
logp_pmn = min(llpmn) + logwt
logp_fmn = min(llfmn) + logwt


logZpmn = logsum(logZpmn,logp_pmn)
logZfmn = logsum(logZfmn,logp_fmn)
logZpc = logsum(logZpc,logp_pc)
logZfc = logsum(logZfc,logp_fc)


for i in range(0, npoints):
    logZfcrem = logsum(llfc[i], logZfcrem)
    logZpcrem = logsum(llpc[i], logZpcrem)
    logZfmnrem = logsum(llfmn[i], logZfmnrem)
    logZpmnrem = logsum(llpmn[i], logZpmnrem)

logZfcrem = logZfcrem + logwt
logZpcrem = logZpcrem + logwt
logZfmnrem = logZfmnrem + logwt
logZpmnrem = logZpmnrem + logwt


logZpc,min(llpc),logZfc,min(llfc),logZpmn,min(llpmn),logZfmn,min(llfmn)

/tmp/ipykernel_125029/1627385975.py:16: RuntimeWarning: divide by zero encountered in log
  logZpcrem = np.log(0)
/tmp/ipykernel_125029/1627385975.py:17: RuntimeWarning: divide by zero encountered in log
  logZfcrem = np.log(0)
/tmp/ipykernel_125029/1627385975.py:18: RuntimeWarning: divide by zero encountered in log
  logZpmnrem = np.log(0)
/tmp/ipykernel_125029/1627385975.py:19: RuntimeWarning: divide by zero encountered in log
  logZfmnrem = np.log(0)
/tmp/ipykernel_125029/1627385975.py:22: RuntimeWarning: divide by zero encountered in log
  logZpc = np.log(0)
/tmp/ipykernel_125029/1627385975.py:23: RuntimeWarning: divide by zero encountered in log
  logZfc = np.log(0)
/tmp/ipykernel_125029/1627385975.py:24: RuntimeWarning: divide by zero encountered in log
  logZpmn = np.log(0)
/tmp/ipykernel_125029/1627385975.py:25: RuntimeWarning: divide by zero encountered in log
  logZfmn = np.log(0)


(-302207.8427986498,
 -302203.22767813294,
 -13753.485852248334,
 -13748.870731731493,
 -109342.42699556862,
 -109337.81187505178,
 -15141.174919961448,
 -15136.559799444607)

In [59]:
# initial stepsize
step = 0

# int index of the loop
j=1

# tolerance value for stopping condition
tol = np.log(10**(-4))

# lowest likelihood in each iteration
lstarpc = []

# log weight
logwt = -np.log(npoints + 1)

# posterior avergae and posterior samples
lpostavgpc = np.zeros(4)
lpostpc = np.zeros(4)

# posterior 2nd moment and posterior 2nd moment samples
post2mavgpc = np.zeros(4)
post2mpc = np.zeros(4)

#information to calculate error
Hpc = 0

#stopping ratio calculator
Rpc = logZpcrem - logZpc

if Rpc < tol:
    print("term1 stop")
    print(logZpc)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglpc = [noisylikelihoodcalc(p_data, 1, *walkers_pc[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarpc.append(min(loglpc))

    # walker index with minimum loglikelihood
    ppc = np.argmin(loglpc)

    # previous iterations log Evidence
    prevlogZpc = logZpc

    # previous iteration's H(information)
    prevHpc = Hpc
    
    # log Z finding
    logZpc = logsum(logZpc, (lstarpc[-1] + logwt))

    # posterior sample
    lpostpc = (walkers_pc[ppc])*np.exp(lstarpc[-1])*np.exp(logwt)

    #information calculation
    Hpc += np.exp(logwt)*np.exp(lstarpc[-1])*lstarpc[-1]

    print("Information")
    print(Hpc)

    # posterior 2nd moment sample
    post2mpc = ((walkers_pc[ppc]) ** 2) * np.exp(lstarpc[-1]) * np.exp(logwt)

    # posterior summation after each iteration
    lpostavgpc = lpostavgpc + lpostpc

    # posterior 2nd moment average
    post2mavgpc = post2mavgpc + post2mpc
    
    print("posterior");
    print(lpostavgpc);
    
    print("Evidence");
    print(logZpc);

    # init Zremaining for the jth iteration
    logZpcrem = np.log(0)

    # calculate Zremaining for the jth iteration
    for i in range(0, npoints):
        logZpcrem = logsum(loglpc[i], logZpcrem)
        
    logZpcrem = logZpcrem + logwt

    print( "Zrem =  "+str(logZpcrem));
    print("lowest likelihood = "+str(min(loglpc)))

    # ratio that determines the stopping ratio
    Rpc = logZpcrem - logZpc
    
    print("ratio")
    print(Rpc)

    #break if stopping ratio is less than tolerance
    if Rpc < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_pc[newwalkernumber]

    # walker replacement  
    trace = MCMCwalkerpc(p_data, 1, thetawalker, lstarpc[-1] , Nsteps, step)

    # step modulation in MCMC walker
    # step = trace[0]
    step = 1

    # replacing the worst walker with the copied walker that undergoes a random walk
    walkers_pc[ppc] = trace[1]

# final evidence calulcation
LogfinalZpc = logsum(prevlogZpc, logZpcrem)

LogfinalZpc

Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-302207.15461425856
Zrem =  -213.94468180416112
lowest likelihood = -302203.22767813294
ratio
301993.2099324544
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-301182.3366593575
Zrem =  -213.9546321350143
lowest likelihood = -301177.7016381789
ratio
300968.3820272225
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-267451.7571739454
Zrem =  -184.99361870953888
lowest likelihood = -267447.112202436
ratio
267266.76355523587
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-119709.19641426404
Zrem =  -185.00356904039205
lowest likelihood = -119704.54149242378
ratio
119524.19284522365
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-82439.48538264455
Zrem =  -184.649214709118
lowest likelihood = -82434.82051047344
ratio
82254.83616793543


/tmp/ipykernel_125029/3929159730.py:81: RuntimeWarning: divide by zero encountered in log
  logZpcrem = np.log(0)


Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-76534.1039737649
Zrem =  -184.02470625284343
lowest likelihood = -76529.42915126294
ratio
76350.07926751205
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-72332.33031693926
Zrem =  -183.4337957259131
lowest likelihood = -72327.64554410645
ratio
72148.89652121335
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-39368.686035197126
Zrem =  -183.16329552686037
lowest likelihood = -39363.99131203346
ratio
39185.52273967027
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-35915.2003526004
Zrem =  -183.17324585771354
lowest likelihood = -35910.49567910588
ratio
35732.02710674268
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-25913.155802402933
Zrem =  -183.1831961885667
lowest likelihood = -25908.44117857756
ratio
25729.972606214367
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-19300.870266371916
Zrem =  -183.19314651941988
lowest likelihood = -19296.14569221569
ratio
19117.677119852495
Information
0.0
posterior
[0. 0. 0. 0.]
Evide

-181.63822479556418

In [60]:
# init posterior avg at the jmax iteration
lapc = np.zeros(4)
l2pc = np.zeros(4)
lh = 0
Hpcfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lapc += walkers_pc[i] * np.exp(llpc[i])
    l2pc += (walkers_pc[i]**2) * np.exp(llpc[i])
    lh += llpc[i]*np.exp(llpc[i])
    
lapc *= np.exp(logwt)
l2pc *= np.exp(logwt)
lh *= np.exp(logwt)

# final posterior average
postfinpc = lpostavgpc+lapc

# final posterior 2 moment
post2mfinpc = post2mavgpc+l2pc

# dividing by the evidence of the model to find the posterior
normpostfinpc = postfinpc/np.exp(LogfinalZpc)

# normalizing posterior 2nd moment
normpost2mfinpc = post2mfinpc/np.exp(LogfinalZpc)

# calculating final information
Hpcfin = prevHpc + lh
Hpcfin = Hpcfin/np.exp(LogfinalZpc) - LogfinalZpc
logZerrpc = np.sqrt(Hpcfin/npoints)

# variance and error in the parameters
varpc = normpost2mfinpc - normpostfinpc**2
stddevpc = np.sqrt(varpc)

# printing information
print("Posterior theta = "+ str(normpostfinpc)+" Error = "+str(stddevpc))
print("ln(Evidence) = "+str(LogfinalZpc)+" +/- "+str(logZerrpc))

Posterior theta = [ 0.17203202  0.99758257 -0.27522713  0.        ] Error = [0.02009345 0.05109632 0.05602725 0.        ]
ln(Evidence) = -181.63822479556418 +/- 0.13352867450494377


## Posterior Value using $Model_{pull, clean}$

The Log Evidence is given as:

$$\ln(Z) = 183.9 \pm 0.21$$

The Posterior averages are given as:
- $D_0 = 0.169 \pm 0.004$
- $\alpha = 0.99 \pm 0.024 $
- $f = -0.29 \pm 0.03$

In [61]:
#initial step size
step = 1

# init index
j=1

#tolerance for stopping condition
tol = np.log(10**(-4))

# lowest likelihood for the jth iteration
lstarfc = []

# log wth value for prior mass
logwt = -np.log(npoints + 1)

# posterior average and posterior samples
lpostavgfc = np.zeros(4)
lpostfc = np.zeros(4)

#information
Hfc = 0

# posterior 2nd moment and posterior 2nd moment samples
post2mavgfc = np.zeros(4)
post2mfc = np.zeros(4)

# stopping ratio
Rfc = logZfcrem - logZfc

# stopping condition
if Rfc< tol:
    print(logZfc)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglfc = [noisylikelihoodcalc(p_data, 1, *walkers_fc[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarfc.append(min(loglfc))

    pfc = np.argmin(loglfc)

    # storing the evidence of the previous iteration
    prevlogZfc = logZfc

    #storing the information of the previous iteration
    prevHfc = Hfc
    
    # log Z finding
    logZfc = logsum(logZfc, (lstarfc[-1] + logwt))

    # posterior samples
    lpostfc = (walkers_fc[pfc])*np.exp(lstarfc[-1])*np.exp(logwt)

    #information
    Hfc += np.exp(logwt)*np.exp(lstarfc[-1])*lstarfc[-1]

    #posterior 2m samples
    post2mfc = ((walkers_fc[pfc])**2)*np.exp(lstarfc[-1])*np.exp(logwt)
    
    # posterior average
    lpostavgfc = lpostavgfc + lpostfc

    #posterior 2m average
    post2mavgfc = post2mavgfc + post2mfc

    print("posterior");
    print(lpostavgfc);
    
    print("Evidence");
    print(logZfc);

    # init Z remaining of the jth iteration
    logZfcrem = np.log(0)

    # calculating Z remaning of the jth iteration
    for i in range(0, npoints):
        logZfcrem = logsum(loglfc[i], logZfcrem)
        
    logZfcrem = logZfcrem + logwt

    print( "Zrem =  "+str(logZfcrem));
    print("lowest likelihood = "+str(min(loglfc)))

    # ratio that determines the stopping ratio
    Rfc = logZfcrem - logZfc
    
    print("ratio")
    print(Rfc)

    #stopping condition and breaks loop
    if Rfc < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    # thetawalker = np.array(walkers_fc[newwalkernumber])

    # walker replacement  
    trace = MCMCwalkerfc(p_data, 1, thetawalker, lstarfc[-1] , Nsteps, 1)

    # step modulation for the random walk
    step = trace[0]

    # replacement of the worst walker with walker performing random walk
    walkers_fc[pfc] = trace[1]
    walkers_fc[pfc][2] = 0
    walkers_fc[pfc][3] = 0

#final Z calculation
LogfinalZfc = logsum(prevlogZfc, logZfcrem)

LogfinalZfc

posterior
[0. 0. 0. 0.]
Evidence
-13752.797667857116
Zrem =  -218.81168324441074
lowest likelihood = -13748.870731731493
ratio
13533.985984612706
posterior
[0. 0. 0. 0.]
Evidence
-13057.047999593568
Zrem =  -188.54998825345865
lowest likelihood = -13052.41297841502
ratio
12868.498011340109
posterior
[0. 0. 0. 0.]
Evidence
-11576.971462769965
Zrem =  -187.84649181233848
lowest likelihood = -11572.326491260565
ratio
11389.124970957626


/tmp/ipykernel_125029/3120566869.py:77: RuntimeWarning: divide by zero encountered in log
  logZfcrem = np.log(0)


posterior
[0. 0. 0. 0.]
Evidence
-10837.5428861679
Zrem =  -187.45611973432858
lowest likelihood = -10832.887964327645
ratio
10650.086766433571
posterior
[0. 0. 0. 0.]
Evidence
-9439.766017981641
Zrem =  -187.17322674663484
lowest likelihood = -9435.101145810535
ratio
9252.592791235007
posterior
[0. 0. 0. 0.]
Evidence
-8228.870528597086
Zrem =  -186.96241714884053
lowest likelihood = -8224.195706095126
ratio
8041.908111448246
posterior
[0. 0. 0. 0.]
Evidence
-7519.403363645279
Zrem =  -186.78871870274062
lowest likelihood = -7514.718590812466
ratio
7332.614644942538
posterior
[0. 0. 0. 0.]
Evidence
-5762.877959122968
Zrem =  -186.65764200271178
lowest likelihood = -5758.183235959301
ratio
5576.220317120256
posterior
[0. 0. 0. 0.]
Evidence
-5753.24238699321
Zrem =  -186.5393220997332
lowest likelihood = -5748.537778862655
ratio
5566.703064893477
posterior
[0. 0. 0. 0.]
Evidence
-5383.017356498959
Zrem =  -186.43240342520184
lowest likelihood = -5378.302732673586
ratio
5196.584953073757


-184.95189768612605

In [62]:
# init posterior avg at the jmax iteration
lafc = np.zeros(4)
l2fc = np.zeros(4)
lahfc = 0

Hfcfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lafc += walkers_fc[i] * np.exp(llfc[i])
    l2fc += (walkers_fc[i]**2) * np.exp(llfc[i])
    lahfc += llfc[i]*np.exp(llfc[i])
    
lafc *= np.exp(logwt)
l2fc *= np.exp(logwt)
lahfc *= np.exp(logwt)

#final posterior calculation
postfinfc = lpostavgfc+lafc
post2mfinfc = post2mavgfc+l2fc

# dividing the posterior by the evidence
normpostfinfc = postfinfc/np.exp(LogfinalZfc)
normpost2mfinfc = post2mfinfc/np.exp(LogfinalZfc)

# information final calculation
Hfcfin = prevHfc + lahfc
Hfcfin = Hfcfin/np.exp(LogfinalZfc) - LogfinalZfc
logZerrfc = np.sqrt(Hfcfin/npoints)

# variance and eror of posterior parameters
varfc = normpost2mfinfc - normpostfinfc**2
stddevfc = np.sqrt(varfc)

# printing information
print("Posterior theta = "+ str(normpostfinfc)+" Error = "+str(stddevfc))
print("ln(Evidence) = "+str(LogfinalZfc)+" +/- "+str(logZerrfc))

Posterior theta = [0.1657674  1.02992421 0.         0.        ] Error = [0.00682641 0.02454162 0.         0.        ]
ln(Evidence) = -184.95189768612605 +/- 0.10124027813142548


# Nested sampling code for noisy models

In [111]:
# # steps for the random walkers in the MCMC algo
# Nsteps = 10000

# # no of walkers considered
# npoints = 100

# # init likelihood of pull, clean and free, clean models
# llpmn = np.zeros(npoints)
# llfmn = np.zeros(npoints)

# # walkers pull clean and walker free clean
# walkers_pmn = np.array(walkergen(npoints, theta_max_pmn,theta_min_pmn))
# walkers_fmn = np.array(walkergen(npoints, theta_max_fmn,theta_min_fmn))

# # print(walkers_fmn)
# # print(walkers_pmn)

# # likelihood  of Mpmn and Mfmn
# for i in range(0,npoints):
#     llpmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_pmn[i])
#     llfmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_fmn[i])


# # wt = 1/(npoints + 1)
# logwt = -np.log(npoints + 1)

# #Zpmn is Z pull clean
# #Zfmn is Z free clean

# # these are the remainders in the jth iteration
# logZpmnrem = np.log(0)
# logZfmnrem = np.log(0)

# # these are the Log Evidences for the respective models
# logZpmn = np.log(0)
# logZfmn = np.log(0)


# logp_pmn = min(llpmn) + logwt
# logp_fmn = min(llfmn) + logwt

# logZpmn = logsum(logZpmn,logp_pmn)
# logZfmn = logsum(logZfmn,logp_fmn)


# for i in range(0, npoints):
#     logZfmnrem = logsum(llfmn[i], logZfmnrem)
#     logZpmnrem = logsum(llpmn[i], logZpmnrem)

# logZfmnrem = logZfmnrem + logwt
# logZpmnrem = logZpmnrem + logwt

# logZpmn,min(llpmn),logZfmn,min(llfmn)

/tmp/ipykernel_63131/1179302201.py:31: RuntimeWarning: divide by zero encountered in log
  logZpmnrem = np.log(0)
/tmp/ipykernel_63131/1179302201.py:32: RuntimeWarning: divide by zero encountered in log
  logZfmnrem = np.log(0)
/tmp/ipykernel_63131/1179302201.py:35: RuntimeWarning: divide by zero encountered in log
  logZpmn = np.log(0)
/tmp/ipykernel_63131/1179302201.py:36: RuntimeWarning: divide by zero encountered in log
  logZfmn = np.log(0)


(-316139.2501793184,
 -316134.63505880156,
 -12067.878821259246,
 -12063.263700742405)

In [63]:
# initial stepsize
step = 1

# int index of the loop
j=1

# tolerance value for stopping condition
tol = np.log(10**(-4))

# lowest likelihood in each iteration
lstarpmn = []

# log weight
logwt = -np.log(npoints + 1)

# posterior avergae and posterior samples
lpostavgpmn = np.zeros(4)
lpostpmn = np.zeros(4)

# posterior 2nd moment and posterior 2nd moment samples
post2mavgpmn = np.zeros(4)
post2mpmn = np.zeros(4)

#information to calculate error
Hpmn = 0

#stopping ratio calculator
Rpmn = logZpmnrem - logZpmn

if Rpmn < tol:
    print("term1 stop")
    print(logZpmn)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglpmn = [noisylikelihoodcalc(p_data, 1, *walkers_pmn[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarpmn.append(min(loglpmn))

    # walker index with minimum loglikelihood
    ppmn = np.argmin(loglpmn)

    # previous iterations log Evidence
    prevlogZpmn = logZpmn

    # previous iteration's H(information)
    prevHpmn = Hpmn
    
    # log Z finding
    logZpmn = logsum(logZpmn, (lstarpmn[-1] + logwt))

    # posterior sample
    lpostpmn = (walkers_pmn[ppmn])*np.exp(lstarpmn[-1])*np.exp(logwt)

    #information calculation
    Hpmn += np.exp(logwt)*np.exp(lstarpmn[-1])*lstarpmn[-1]

    print("Information")
    print(Hpmn)

    # posterior 2nd moment sample
    post2mpmn = ((walkers_pmn[ppmn]) ** 2) * np.exp(lstarpmn[-1]) * np.exp(logwt)

    # posterior summation after each iteration
    lpostavgpmn = lpostavgpmn + lpostpmn

    # posterior 2nd moment average
    post2mavgpmn = post2mavgpmn + post2mpmn
    
    print("posterior");
    print(lpostavgpmn);
    
    print("Evidence");
    print(logZpmn);

    # init Zremaining for the jth iteration
    logZpmnrem = np.log(0)

    # calculate Zremaining for the jth iteration
    for i in range(0, npoints):
        logZpmnrem = logsum(loglpmn[i], logZpmnrem)
        
    logZpmnrem = logZpmnrem + logwt

    print( "Zrem =  "+str(logZpmnrem));
    print("lowest likelihood = "+str(min(loglpmn)))

    # ratio that determines the stopping ratio
    Rpmn = logZpmnrem - logZpmn
    
    print("ratio")
    print(Rpmn)

    #break if stopping ratio is less than tolerance
    if Rpmn < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_pmn[newwalkernumber]

    # walker replacement  
    trace = MCMCwalker(p_data, 1, thetawalker, lstarpmn[-1] , Nsteps, step)

    # step modulation in MCMC walker
    step = 1

    # replacing the worst walker with the copied walker that undergoes a random walk
    walkers_pmn[ppmn] = trace[1]

# final evidence calulcation
LogfinalZpmn = logsum(prevlogZpmn, logZpmnrem)

LogfinalZpmn

Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-109341.73881117739
Zrem =  -259.2752524360088
lowest likelihood = -109337.81187505178
ratio
109082.46355874138
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-89400.88471380917
Zrem =  -259.28520276686197
lowest likelihood = -89396.24969263062
ratio
89141.59951104231
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-88241.91777582321
Zrem =  -259.2951530977152
lowest likelihood = -88237.2728043138
ratio
87982.6226227255
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-65148.5745356021
Zrem =  -259.3051034285683
lowest likelihood = -65143.91961376185
ratio
64889.26943217353


/tmp/ipykernel_125029/178806207.py:81: RuntimeWarning: divide by zero encountered in log
  logZpmnrem = np.log(0)


Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-49141.0485711736
Zrem =  -256.6289257277655
lowest likelihood = -49136.38369900249
ratio
48884.41964544583
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-23294.133665220357
Zrem =  -187.9247330716793
lowest likelihood = -23289.458842718395
ratio
23106.20893214868
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-21012.39798745692
Zrem =  -187.93468340253244
lowest likelihood = -21007.713214624106
ratio
20824.46330405439
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-19498.740584477942
Zrem =  -187.94463373338561
lowest likelihood = -19494.045861314276
ratio
19310.795950744556
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-18430.241424094253
Zrem =  -187.9545840642388
lowest likelihood = -18425.53675059973
ratio
18242.286840030014
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-14556.447384465431
Zrem =  -187.96453439509196
lowest likelihood = -14551.732760640058
ratio
14368.482850070339
Information
0.0
posterior
[0. 0. 0. 0.]

-183.61711287093766

In [64]:
# init posterior avg at the jmax iteration
lapmn = np.zeros(4)
l2pmn = np.zeros(4)
lhmn = 0
Hpmnfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lapmn += walkers_pmn[i] * np.exp(llpmn[i])
    l2pmn += (walkers_pmn[i]**2) * np.exp(llpmn[i])
    lh += llpmn[i]*np.exp(llpmn[i])
    
lapmn *= np.exp(logwt)
l2pmn *= np.exp(logwt)
lhmn *= np.exp(logwt)

# final posterior average
postfinpmn = lpostavgpmn+lapmn

# final posterior 2 moment
post2mfinpmn = post2mavgpmn+l2pmn

# dividing by the evidence of the model to find the posterior
normpostfinpmn = postfinpmn/np.exp(LogfinalZpmn)

# normalizing posterior 2nd moment
normpost2mfinpmn = post2mfinpmn/np.exp(LogfinalZpmn)

# calculating final information
Hpmnfin = prevHpmn + lhmn
Hpmnfin = Hpmnfin/np.exp(LogfinalZpmn) - LogfinalZpmn
logZerrpmn = np.sqrt(Hpmnfin/npoints)

# variance and error in the parameters
varpmn = normpost2mfinpmn - normpostfinpmn**2
stddevpmn = np.sqrt(varpmn)

# printing information
print("Posterior theta = "+ str(normpostfinpmn)+" Error = "+str(stddevpmn))
print("ln(Evidence) = "+str(LogfinalZpmn)+" +/- "+str(logZerrpmn))

Posterior theta = [ 0.14978286  1.0474086  -0.28722101  0.04982687] Error = [0.04442118 0.10226255 0.08027254 0.05205642]
ln(Evidence) = -183.61711287093766 +/- 0.16925058513204794


## Posterior Value using $Model_{pull, clean}$

The Log Evidence is given as:

$$\ln(Z) = 183.9 \pm 0.21$$

The Posterior averages are given as:
- $D_0 = 0.169 \pm 0.004$
- $\alpha = 0.99 \pm 0.024 $
- $f = -0.29 \pm 0.03$

In [65]:
#initial step size
step = 1

# init index
j=1

#tolerance for stopping condition
tol = np.log(10**(-4))

# lowest likelihood for the jth iteration
lstarfmn = []

# log wth value for prior mass
logwt = -np.log(npoints + 1)

# posterior average and posterior samples
lpostavgfmn = np.zeros(4)
lpostfmn = np.zeros(4)

#information
Hfmn = 0

# posterior 2nd moment and posterior 2nd moment samples
post2mavgfmn = np.zeros(4)
post2mfmn = np.zeros(4)

# stopping ratio
Rfmn = logZfmnrem - logZfmn

# stopping condition
if Rfmn< tol:
    print(logZfmn)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglfmn = [noisylikelihoodcalc(p_data, 1, *walkers_fmn[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarfmn.append(min(loglfmn))

    pfmn = np.argmin(loglfmn)

    # storing the evidence of the previous iteration
    prevlogZfmn = logZfmn

    #storing the information of the previous iteration
    prevHfmn = Hfmn
    
    # log Z finding
    logZfmn = logsum(logZfmn, (lstarfmn[-1] + logwt))

    # posterior samples
    lpostfmn = (walkers_fmn[pfmn])*np.exp(lstarfmn[-1])*np.exp(logwt)

    #information
    Hfmn += np.exp(logwt)*np.exp(lstarfmn[-1])*lstarfmn[-1]

    #posterior 2m samples
    post2mfmn = ((walkers_fmn[pfmn])**2)*np.exp(lstarfmn[-1])*np.exp(logwt)
    
    # posterior average
    lpostavgfmn = lpostavgfmn + lpostfmn

    #posterior 2m average
    post2mavgfmn = post2mavgfmn + post2mfmn

    print("posterior");
    print(lpostavgfmn);
    
    print("Evidence");
    print(logZfmn);

    # init Z remaining of the jth iteration
    logZfmnrem = np.log(0)

    # calculating Z remaning of the jth iteration
    for i in range(0, npoints):
        logZfmnrem = logsum(loglfmn[i], logZfmnrem)
        
    logZfmnrem = logZfmnrem + logwt

    print( "Zrem =  "+str(logZfmnrem));
    print("lowest likelihood = "+str(min(loglfmn)))

    # ratio that determines the stopping ratio
    Rfmn = logZfmnrem - logZfmn
    
    print("ratio")
    print(Rfmn)

    #stopping condition and breaks loop
    if Rfmn < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_fmn[newwalkernumber]

    # walker replacement  
    trace = MCMCwalkerfmn(p_data, 1, thetawalker, lstarfmn[-1] , Nsteps, step)

    # step modulation for the random walk
    step = 1

    # replacement of the worst walker with walker performing random walk
    walkers_fmn[pfmn] = trace[1]

#final Z calculation
LogfinalZfmn = logsum(prevlogZfmn, logZfmnrem)

LogfinalZfmn

posterior
[0. 0. 0. 0.]
Evidence
-15140.48673557023
Zrem =  -278.14216340630804
lowest likelihood = -15136.559799444607
ratio
14862.344572163922
posterior
[0. 0. 0. 0.]
Evidence
-9472.326121571834
Zrem =  -189.13964579122575
lowest likelihood = -9467.691100393286
ratio
9283.186475780609


/tmp/ipykernel_125029/970416612.py:77: RuntimeWarning: divide by zero encountered in log
  logZfmnrem = np.log(0)


posterior
[0. 0. 0. 0.]
Evidence
-8254.893097356573
Zrem =  -188.4741564781158
lowest likelihood = -8250.248125847173
ratio
8066.418940878457
posterior
[0. 0. 0. 0.]
Evidence
-7204.719602988114
Zrem =  -187.83204004627896
lowest likelihood = -7200.06468114786
ratio
7016.887562941835
posterior
[0. 0. 0. 0.]
Evidence
-6829.718594933909
Zrem =  -187.36304808071156
lowest likelihood = -6825.053722762802
ratio
6642.355546853198
posterior
[0. 0. 0. 0.]
Evidence
-6434.002760462925
Zrem =  -187.37299841156474
lowest likelihood = -6429.327937960965
ratio
6246.629762051361
posterior
[0. 0. 0. 0.]
Evidence
-6132.228353820034
Zrem =  -187.24229169610058
lowest likelihood = -6127.543580987221
ratio
5944.986062123933
posterior
[0. 0. 0. 0.]
Evidence
-6116.393906237175
Zrem =  -187.0276404794238
lowest likelihood = -6111.699183206304
ratio
5929.366265757751
posterior
[0. 0. 0. 0.]
Evidence
-5358.436063380444
Zrem =  -186.9140756435691
lowest likelihood = -5353.731389885924
ratio
5171.5219877368745
po

-185.58604914665486

In [66]:
# init posterior avg at the jmax iteration
lafmn = np.zeros(4)
l2fmn = np.zeros(4)
lahfmn = 0

Hfmnfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lafmn += walkers_fmn[i] * np.exp(llfmn[i])
    l2fmn += (walkers_fmn[i]**2) * np.exp(llfmn[i])
    lahfmn += llfmn[i]*np.exp(llfmn[i])
    
lafmn *= np.exp(logwt)
l2fmn *= np.exp(logwt)
lahfmn *= np.exp(logwt)

#final posterior calculation
postfinfmn = lpostavgfmn+lafmn
post2mfinfmn = post2mavgfmn+l2fmn

# dividing the posterior by the evidence
normpostfinfmn = postfinfmn/np.exp(LogfinalZfmn)
normpost2mfinfmn = post2mfinfmn/np.exp(LogfinalZfmn)

# information final calculation
Hfmnfin = prevHfmn + lahfmn
Hfmnfin = Hfmnfin/np.exp(LogfinalZfmn) - LogfinalZfmn
logZerrfmn = np.sqrt(Hfmnfin/npoints)

# variance and eror of posterior parameters
varfmn = normpost2mfinfmn - normpostfinfmn**2
stddevfmn = np.sqrt(varfmn)

# printing information
print("Posterior theta = "+ str(normpostfinfmn)+" Error = "+str(stddevfmn))
print("ln(Evidence) = "+str(LogfinalZfmn)+" +/- "+str(logZerrfmn))

Posterior theta = [0.14588788 1.08462895 0.         0.03628848] Error = [0.02810318 0.07244701 0.         0.02985658]
ln(Evidence) = -185.58604914665486 +/- 0.11543631467552566


In [35]:
# seeing the difference of the Evidence
print("log(Zpc) - log(Zfc) = "+ str(LogfinalZpc - LogfinalZfc )+" +/- " +str(logZerrfc + logZerrpc))

log(Zpc) - log(Zfc) = 2.7534700196888195 +/- 0.2911618340020794


In [67]:
# Model probability of the models considered
mpc = np.exp(LogfinalZpc)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))
mfc = np.exp(LogfinalZfc)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))
mpmn = np.exp(LogfinalZpmn)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))
mfmn = np.exp(LogfinalZfmn)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))

print("Model prob for Pull clean = " + str(mpc)[:5])
print("Model prob for free clean = " + str(mfc)[:5])
print("Model prob for Pull mn = " + str(mpmn)[:5])
print("Model prob for free mn = " + str(mfmn)[:5])

Model prob for Pull clean = 0.837
Model prob for free clean = 0.030
Model prob for Pull mn = 0.115
Model prob for free mn = 0.016


In [36]:
# ratio of evidences considered
np.exp(LogfinalZpc)/(np.exp(LogfinalZfc))

15.697006410589392

In [70]:
otpt = []
header = "========================================================================================================== \n"
header +="                                RESULTS FROM NESTED SAMPLING                                               \n"
header +="========================================================================================================== \n"
otpt.append(header)
lin1 = ','.join(str(i) for i in p_data)
lin1 = "The raw position data =  ["+lin1+"]  \n\n\n"
otpt.append(lin1)
linearr = [0,0,0,0]
MODELS = ["(free, clean)","(pull, clean)","(free, mn)","(pull, mn)"]
MODELPROB = [mfc,mpc,mfmn,mpmn]
MODELZ = [LogfinalZfc,LogfinalZpc, LogfinalZfmn, LogfinalZpmn]
MODELZERR = [logZerrfc, logZerrpc, logZerrfmn, logZerrpmn]
MODELPARAM = [normpostfinfc, normpostfinpc, normpostfinfmn, normpostfinpmn]
MODELPARAMERR = [stddevfc, stddevpc, stddevfmn, stddevpmn]
MODELINFO = [Hfcfin, Hpcfin, Hfmnfin, Hpmnfin]
MODELMLE = [max(loglfc), max(loglpc), max(loglfmn), max(loglpmn)]
for i in range(0,4):
    linearr[i] = f"Model %d  %s  with Model Prob = %.3f \n"%(i+1,MODELS[i],MODELPROB[i])
    linearr[i] += f"ln Evidence = %.3f  +/- %.5f \n"%(MODELZ[i], MODELZERR[i])
    linearr[i] += f"Posterior Paramters \n"
    linearr[i] += f" 1. Diffusion Coefficient D_0 = %.3f  +/-  %.4f \n"%(MODELPARAM[i][0],3*MODELPARAMERR[i][0])
    linearr[i] += f" 2. Exponent alpha            = %.3f  +/-  %.4f \n"%(MODELPARAM[i][1],3*MODELPARAMERR[i][1])
    linearr[i] += f" 3. Applied Force f           = %.3f  +/-  %.4f \n"%(MODELPARAM[i][2],3*MODELPARAMERR[i][2])
    linearr[i] += f" 4. Var of Measurement noise  = %.3f  +/-  %.4f \n"%(MODELPARAM[i][3],3*MODELPARAMERR[i][3])
    linearr[i] += f" Maximum Likelihood = %.3f \n"%(MODELMLE[i])
    linearr[i] += f" Information H = %.5f  \n\n"%(MODELINFO[i])
for i in range(0,4):
    otpt.append(linearr[i])
hp = f"Highest Model probability for the given data is for Model %s \n\n\n\n"%(MODELS[MODELZ.index(max(MODELZ))])
otpt.append(hp)

In [71]:
file = open("NScleantrackKrog.txt", "w")
file.writelines(otpt)
file.close()

# irrelevant shit to test my moronic code

In [170]:
N=500000
a = walkergen(1,theta_max_pmn,theta_min_pmn)[0]
lla = noisylikelihoodcalc(p_data, 1, *a)
a,lla

(array([69.05804744, -0.62777981,  0.39170489,  6.3841076 ]),
 -662.123144272124)

In [76]:
a = walkergen(1,theta_max_pmn,theta_min_pmn)[0]
a[0] = jeff(a[0], D_min, D_max)
lla = noisylikelihoodcalc(p_data, 1, *a)
print("init_Walker = " + str(a))
print("init likelihood = " +str(lla))
t = MCMCwalker(p_data, 1, a, lla, N, 1)
step = t[0]
fp = t[1]
llf = noisylikelihoodcalc(p_data, 1, fp[0], fp[1], fp[2], fp[3])
# fp
# t[1]
llf,fp

init_Walker = [ 3.56741994e+265 -5.17185102e-001 -4.85344406e-001  4.66494122e+001]
init likelihood = -inf


NameError: name 'N' is not defined

In [4]:
p_data = np.array([50.0,50.419970251583734,45.6568431381102,40.153500472839376,43.921288092176965,44.20413351437964,42.22491100428267,44.07718270189529,38.37371229570785,35.041415065824616,30.57815817618083,25.255912228927436,23.284250387841105,21.0914991105031,20.505144524001363,16.047609051036495,19.23318306843276,17.090351802897743,25.03279028115675,29.37941086339806,28.710167677194764,32.65764239618771,33.11108218518014,31.74849850197023,28.779966213851992,28.15869540902125,31.096505291847066,32.14969145182582,35.555585192994364,31.78382757178809,31.176547571379015,18.871828817666724,16.920921301699906,17.900402515103462,24.400822847254496,21.062383513467054,10.572435193045767,12.048800075488982,13.92033275934067,13.837571736684223,18.82098820779177,16.434193993028302,18.407342830124136,19.44214454486823,16.42278506295819,17.042712756335035,17.202417773998327,17.890658100209496,16.070057233414097,20.70966427897566,21.13216478721364,20.792074502429525,20.222317308828693,19.913143466173196,24.97543323695711,21.014140450488437,18.665527626231345,18.77148773197088,18.919413982714612,23.093609677986546,26.663811154539385,28.82865582494256,24.994188027698247,22.43198849196611,17.973229645235516,14.166727208608435,11.139484847049985,13.208216463072597,12.926778719734504,13.300204556214084,14.291828480870437,18.98904144905074,15.28324353989863,15.019784292904504,12.56512012672789,16.494474042725386,14.907297273776889,11.326550509088596,8.916467172594722,10.541863498103691,9.280406081460534,10.404687187965795,10.306817640171193,13.443219889141204,11.625004899787799,9.117736822397429,8.63146779695835,7.4579341620111625,6.497883493655436,7.635973020870801,8.997338916097979,7.8391596109066795,6.629439540519379,6.900255518044078,7.095977196774567,7.870260500712156,6.970146377861853,7.241067145870732,6.829087193790995,3.0390056992038854])
llt = noisylikelihoodcalc(p_data, 1, 0.2, 1,-0.3,0)
llt

-250.37361748663187